<a href="https://colab.research.google.com/github/muralidharan-1708/LLM-Projects/blob/main/VectorDB_Faiss_CPU_Sentence_Transformers_Using_Gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install faiss-cpu sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 74.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

In [ ]:
# Example Sentences for knowledge base (like open source)
sentences = ["I love cats",
             "Dogs are the Best",
             "Artificial Intelligence is the future",
             "Cats are cute animals",
             "I enjoy machine learning",
             "I have a pet dog"
             ]

#Load the pre-trained Model
model = SentenceTransformer('all-MiniLM-L6-v2') #Fast conversion text into embeddings

In [ ]:
embeddings = model.encode(sentences)
embeddings = np.array(embeddings).astype('float32')

In [ ]:
embeddings.shape

(6, 384)

In [ ]:
#find dimensions for embeddings
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension) #faiss - vectorDB --> Give index for all 384 vectors using Euclidean distance
index.add(embeddings)

In [ ]:
query = ['I adore Kittens', 'chatgpt uses AI']
query_vector = model.encode(query).astype("float32")
top_k = 3
distances, indices = index.search(query_vector, top_k)

print("Query:", query)
print("\nTop matches : ")
for i in range(len(query)):
  print(f"\nMatches for '{query[i]}':")
  for idx in indices[i]:  #index[i] --> gets the top_k similarity for each query , here we are using 2 query
    print("-",sentences[idx])

Query: ['I adore Kittens', 'chatgpt uses AI']

Top matches : 

Matches for 'I adore Kittens':
- I love cats
- Cats are cute animals
- Dogs are the Best

Matches for 'chatgpt uses AI':
- Artificial Intelligence is the future
- I enjoy machine learning
- Dogs are the Best


In [ ]:
!pip install faiss-cpu sentence-transformers openai

In [ ]:
import openai
gemini_api_key = "ENTER YOUR API_KEY"

In [ ]:
context = ""
for i in range(len(query)):
  for idx in indices[i]:
    context += sentences[idx]


    prompt = f"""
            Answer the question based on the context below :
            context :
            {context}

            Question : {query[i]}
            Answer :
    """

    import google.generativeai as genai

    genai.configure(api_key = gemini_api_key)


    model = genai.GenerativeModel("gemini-1.5-flash")  # or "gemini-1.5-pro"

    response = model.generate_content(prompt)

    print(response.text)

The provided context only states "I love cats".  While this implies a positive feeling towards cats in general, it doesn't explicitly confirm a love for *kittens*.  Therefore, the answer is:  **Uncertain/Cannot be determined.**

The provided text states "I love cats".  Since kittens are cats, the answer is **Yes**.

The provided text doesn't directly address kittens.  While it states "I love cats,"  it doesn't confirm or deny a preference for kittens specifically.  Therefore, the answer is **uncertain** or **cannot be determined**.

The provided text states that "Artificial Intelligence is the future".  Since ChatGPT is an AI, the answer is **yes**.

Yes

Yes

